In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import glob
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import numpy as np
from cycler import cycler
%matplotlib inline

#from sklearn import linear_model
from scipy import stats

In [2]:
t_titre = 35
t_label = 25

In [3]:
liste = []
for fname in glob.glob('data/*pkl'):
    a = fname.split('/')[1].split('.')[0].split('_')
    liste.append(a)
    print(a)

['enregistrement', 'IP', '2017-09-28', '115250']
['pari', 'FM', '2017-09-26', '143606']
['enregistrement', 'BMC', '2017-09-26', '095637']
['pari', 'BMC', '2017-09-25', '142503']
['pari', 'LB', '2017-09-19', '152205']
['pari', 'CS', '2017-09-28', '154353']
['enregistrement', 'LB', '2017-09-20', '151043']
['enregistrement', 'DC', '2017-09-27', '161040']
['pari', 'IP', '2017-09-26', '122431']
['pari', 'DC', '2017-09-26', '162348']


In [4]:
exp = []
for x in range(len(liste)) :
    if liste[x][0]=='pari' and liste[x][1] in ['BMC','CS','DC','FM','IP','LB'] :
        a = 'data/%s_%s_%s_%s.pkl'%(liste[x][0], liste[x][1],liste[x][2],liste[x][3])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            exp.append(b)

In [5]:
param = []
for x in range(len(liste)) :
    if liste[x][0]=='pari' and liste[x][1] in ['BMC','CS','DC','FM','IP','LB'] :
        a = 'parametre/%s_param_Fit.pkl'%(liste[x][1])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            b['observer'] = liste[x][1]
            param.append(b)

FileNotFoundError: [Errno 2] No such file or directory: 'parametre/FM_param_Fit.pkl'

In [ ]:
N_blocks = 3
N_trials = 200
full_proba = []
full_results = []
full_va = []
full_bino = []
for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    p = exp[x]['p']
    proba = p[:,:,1]
    bino = p[:,:,0]
    v_anti = param[x]['v_anti']
    results = (exp[x]['results']+1)/2
    for block in range(N_blocks) :
        for trial in range(N_trials):
            full_proba.append(proba[trial, block])
            full_results.append(results[trial, block])
            full_va.append(v_anti[block][trial])
            full_bino.append(bino[trial, block])

## Probabilité réel

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/1.6180))
axs.axis([-0.1, 1.1, -0.1, 1.1])
for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):

    p = exp[x]['p']
    results = (exp[x]['results']+1)/2

    
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(p[:,:,1], results, c=color, alpha=0.5, linewidths=0)

slope, intercept, r_value, p_value, std_err = stats.linregress(full_proba, full_results)
x_test = np.linspace(np.min(full_proba), np.max(full_proba), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title("Real Probability vs Probability Bet", fontsize=t_titre)
plt.xlabel('Real Probability', fontsize=t_label)
plt.ylabel('Probability Bet', fontsize=t_label)

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/(1.6180)))
axs.axis([-0.1, 1.1, -5, 5])

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    p = exp[x]['p']
    v_anti_0_1 = param[x]['v_anti']
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(p[:,:,1], v_anti_0_1, c=color, alpha=0.5, linewidths=0)

slope, intercept, r_value, p_value, std_err = stats.linregress(full_proba, full_va)
x_test = np.linspace(np.min(full_proba), np.max(full_proba), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title("Real Probability vs Acceleration", fontsize=t_titre)
plt.xlabel('Real Probability', fontsize=t_label)
plt.ylabel('Acceleration of anticipation (deg/s$^2$)', fontsize=t_label)

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/(1.6180)))
axs.axis([-0.1, 1.1, -5, 5])

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti_0_1 = param[x]['v_anti']
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(results, v_anti_0_1, c=color, alpha=0.5, linewidths=0)

slope, intercept, r_value, p_value, std_err = stats.linregress(full_results, full_va)
x_test = np.linspace(np.min(full_results), np.max(full_results), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)


plt.title("Probability Bet vs Acceleration", fontsize=t_titre)
plt.xlabel('Probability Bet', fontsize=t_label)
plt.ylabel('Acceleration of anticipation (deg/s$^2$)', fontsize=t_label)

## Probabilité estimé

In [ ]:
def moving_ave(list_D, tau, p_init=.5, delta=0):
    """
    This function is an agent that infer a P contengencie in function of a pre-established list_D of
    either rightward or leftward trials. p_init means we start from a prior of 50% chance for a target
    to go to each size. Delta = roll
    """
    list_p = p_init*np.ones_like(list_D)
    for i, D in enumerate(list_D[:-1]):
        list_p[i+1] = (1 -  1/tau)*list_p[i] + 1/tau * D
    return np.roll(list_p, delta).flatten()
ploup = exp[0]['p']
plt.plot(moving_ave(ploup[:,:,0], tau=5, delta=2), c='r')

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/1.6180))
axs.axis([-0.1, 1.1, -0.1, 1.1])
for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):

    p = exp[x]['p']
    results = (exp[x]['results']+1)/2 
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(moving_ave(p[:,:,0], tau=5, delta=2), results, c=color, alpha=0.5,linewidths=0)
    
slope, intercept, r_value, p_value, std_err = stats.linregress(moving_ave(full_bino, tau=5, delta=2), full_results)
x_test = np.linspace(np.min(moving_ave(full_bino, tau=5, delta=2)), np.max(moving_ave(full_bino, tau=5, delta=2)), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title(r'Estimated Probability vs Probability Bet', fontsize=t_titre)
plt.xlabel(r'Estimated Probability', fontsize=t_label)
plt.ylabel('Probability Bet', fontsize=t_label)

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/(1.6180)))
axs.axis([-0.1, 1.1, -5, 5])

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    p = exp[x]['p']
    v_anti_0_1 = param[x]['v_anti']
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(moving_ave(p[:,:,0], tau=5, delta=2), v_anti_0_1, c=color, alpha=0.5, linewidths=0)
       
slope, intercept, r_value, p_value, std_err = stats.linregress(moving_ave(full_bino, tau=5, delta=2), full_va)
x_test = np.linspace(np.min(moving_ave(full_bino, tau=5, delta=2)), np.max(moving_ave(full_bino, tau=5, delta=2)), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title("Estimated Probability vs Acceleration", fontsize=t_titre)
plt.xlabel('Estimated Probability', fontsize=t_label)
plt.ylabel('Acceleration of anticipation (deg/s$^2$)', fontsize=t_label)

# - les 15 premier essais après le switch

In [ ]:
proba_switch_x = []
liste_switch_x = []
block_result_switch_x = []
block_v_anti_switch_x = []
proba_x = []
bino_x = []
for x in range(len(exp)):

    N_trials = exp[x]['N_trials']
    N_blocks = exp[x]['N_blocks']
    p = exp[x]['p']
    results = (exp[x]['results']+1)/2
    v_anti = param[x]['v_anti']
    
    liste_switch=[]
    proba_switch = []
    result_switch = []
    v_anti_switch = []
    prob = []
    bi = []
    
    for block in range(N_blocks):

        switch = [0]
        p_switch = [p[0, block, 1]]
        for s in range(N_trials):
            if p[s, block, 2]==1 :
                switch.append(s)
                p_switch.append(p[s, block, 1])
        switch.append(N_trials)

        liste_switch.append(switch)
        proba_switch.append(p_switch)

        liste_result_switch = []
        liste_v_anti_switch = []
        liste_proba = []
        for s1 in range(len(switch)-1) :

            for lala in np.arange(switch[s1]+15, switch[s1+1]) :
                result_switch.append(results[lala, block])
                v_anti_switch.append(v_anti[block][lala])
                prob.append(p[lala, block, 1])
                bi.append(p[lala, block, 0])


    proba_switch_x.append(proba_switch)
    liste_switch_x.append(liste_switch)
    block_result_switch_x.append(result_switch)
    block_v_anti_switch_x.append(v_anti_switch)
    proba_x.append(prob)
    bino_x.append(bi)
    

In [ ]:
full_proba_1 = []
full_results_1 = []
full_va_1 = []
full_bino_1 = []
for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):

    proba = proba_x[x]
    v_anti = block_v_anti_switch_x[x]
    results = block_result_switch_x[x]
    bino = bino_x[x]
    for trial in range(len(proba)):
        full_proba_1.append(proba[trial])
        full_results_1.append(results[trial])
        full_va_1.append(v_anti[trial])
        full_bino_1.append(bino[trial])

## Probabilité réel

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/1.6180))
axs.axis([-0.1, 1.1, -0.1, 1.1])
for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):

    p = proba_x[x]
    results = block_result_switch_x[x] #(exp[x]['results']+1)/2

    
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(p, results, c=color, alpha=0.5, linewidths=0)

slope, intercept, r_value, p_value, std_err = stats.linregress(full_proba_1, full_results_1)
x_test = np.linspace(np.min(full_proba_1), np.max(full_proba_1), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title("Real Probability vs Probability Bet -- 15", fontsize=t_titre)
plt.xlabel('Real Probability', fontsize=t_label)
plt.ylabel('Probability Bet', fontsize=t_label)

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/(1.6180)))
axs.axis([-0.1, 1.1, -5, 5])

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    
    p = proba_x[x]
    v_anti_0_1 = block_v_anti_switch_x[x]
    
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(p, v_anti_0_1, c=color, alpha=0.5, linewidths=0)
    
slope, intercept, r_value, p_value, std_err = stats.linregress(full_proba_1, full_va_1)
x_test = np.linspace(np.min(full_proba_1), np.max(full_proba_1), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title("Real Probability vs Acceleration -- 15", fontsize=t_titre)
plt.xlabel('Real Probability', fontsize=t_label)
plt.ylabel('Acceleration of anticipation (deg/s$^2$)', fontsize=t_label)

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/(1.6180)))
axs.axis([-0.1, 1.1, -5, 5])

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    results = block_result_switch_x[x] #(exp[x]['results']+1)/2
    v_anti_0_1 = block_v_anti_switch_x[x]
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(results, v_anti_0_1, c=color, alpha=0.5, linewidths=0)
     
slope, intercept, r_value, p_value, std_err = stats.linregress(full_results_1, full_va_1)
x_test = np.linspace(np.min(full_results_1), np.max(full_results_1), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)


plt.title("Probability Bet vs Acceleration -- 15", fontsize=t_titre)
plt.xlabel('Probability Bet', fontsize=t_label)
plt.ylabel('Acceleration of anticipation (deg/s$^2$)', fontsize=t_label)

## probabilité estimé

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/1.6180))
axs.axis([-0.1, 1.1, -0.1, 1.1])
for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):

    bino = bino_x[x]
    results = block_result_switch_x[x] #(exp[x]['results']+1)/2

    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(moving_ave(bino, tau=5, delta=2), results, c=color, alpha=0.5, linewidths=0)

slope, intercept, r_value, p_value, std_err = stats.linregress(moving_ave(full_bino_1, tau=5, delta=2), full_results_1)
x_test = np.linspace(np.min(moving_ave(full_bino_1, tau=5, delta=2)), np.max(moving_ave(full_bino_1, tau=5, delta=2)), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title("Estimated Probability vs Probability Bet -- 15", fontsize=t_titre)
plt.xlabel('Estimated Probability', fontsize=t_label)
plt.ylabel('Probability Bet', fontsize=t_label)

In [ ]:
fig_width = 15
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/(1.6180)))
axs.axis([-0.1, 1.1, -5, 5])

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    
    p = proba_x[x]
    v_anti = block_v_anti_switch_x[x]
    
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    plt.scatter(moving_ave(p, tau=5, delta=2), v_anti, c=color, alpha=0.5, linewidths=0)

slope, intercept, r_value, p_value, std_err = stats.linregress(moving_ave(full_proba_1, tau=5, delta=2), full_va_1)
x_test = np.linspace(np.min(moving_ave(full_proba_1, tau=5, delta=2)), np.max(moving_ave(full_proba_1, tau=5, delta=2)), 100)
fitLine = slope * x_test + intercept
plt.plot(x_test, fitLine, c='k', linewidth=2)

plt.title("Estimated Probability vs Acceleration -- 15", fontsize=t_titre)
plt.xlabel('Estimated Probability', fontsize=t_label)
plt.ylabel('Acceleration of anticipation (deg/s$^2$)', fontsize=t_label)

# Autre Test

In [ ]:
proba_switch_x = []
liste_switch_x = []
block_result_switch_x = []
block_v_anti_switch_x = []

block_m_result_switch_x = []
block_m_v_anti_switch_x = []
block_std_result_switch_x = []
block_std_v_anti_switch_x = []

for x in range(len(exp)):

    N_trials = exp[x]['N_trials']
    N_blocks = exp[x]['N_blocks']
    p = exp[x]['p']
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti_0_1 = param[x]['v_anti']
    
    
    proba_switch = []
    liste_switch = []
    block_result_switch = []
    block_v_anti_switch = []

    block_m_result_switch = []
    block_m_v_anti_switch = []
    block_std_result_switch = []
    block_std_v_anti_switch = []
    for block in range(N_blocks):

        switch = [0]
        p_switch = [p[0, block, 1]]
        for s in range(N_trials):
            if p[s, block, 2]==1 :
                switch.append(s)
                p_switch.append(p[s, block, 1])
        switch.append(N_trials)

        liste_switch.append(switch)
        proba_switch.append(p_switch)

        liste_result_switch = []
        liste_v_anti_switch = []

        liste_m_result_switch = []
        liste_m_v_anti_switch = []
        liste_std_result_switch = []
        liste_std_v_anti_switch = []
        
        for s1 in range(len(switch)-1) :
            result_switch = results[switch[s1]:switch[s1+1], block]
            v_anti_switch = v_anti_0_1[block][switch[s1]:switch[s1+1]]

            liste_result_switch.append(result_switch)
            liste_v_anti_switch.append(v_anti_switch)

            m_result_switch = np.mean(results[switch[s1]:switch[s1+1], block])
            m_v_anti_switch = np.mean(v_anti_0_1[block][switch[s1]:switch[s1+1]])*100
            
            std_result_switch = np.std(results[switch[s1]:switch[s1+1],block])
            std_v_anti_switch = np.std(v_anti_0_1[block][switch[s1]:switch[s1+1]])*100
            
            liste_m_result_switch.append(m_result_switch)
            liste_m_v_anti_switch.append(m_v_anti_switch)
            liste_std_result_switch.append(std_result_switch)
            liste_std_v_anti_switch.append(std_v_anti_switch)

        block_result_switch.append(liste_result_switch)
        block_v_anti_switch.append(liste_v_anti_switch)

        block_m_result_switch.append(liste_m_result_switch)
        block_m_v_anti_switch.append(liste_m_v_anti_switch)
        block_std_result_switch.append(liste_std_result_switch)
        block_std_v_anti_switch.append(liste_std_v_anti_switch)
    
    proba_switch_x.append(proba_switch)
    liste_switch_x.append(liste_switch)
    block_result_switch_x.append(block_result_switch)
    block_v_anti_switch_x.append(block_v_anti_switch)

    block_m_result_switch_x.append(block_m_result_switch)
    block_m_v_anti_switch_x.append(block_m_v_anti_switch)
    block_std_result_switch_x.append(block_std_result_switch)
    block_std_v_anti_switch_x.append(block_std_v_anti_switch)

In [ ]:
fig_width = 12
fig, axs = plt.subplots(3, 1, figsize=(fig_width, (fig_width*2)/(1.6180)))
fig.subplots_adjust(hspace=0.3)

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):

    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    
    dic = {}
    N_blocks = exp[x]['N_blocks']
    
    for block in range(N_blocks):

        axs[block].errorbar(range(len(proba_switch_x[x][block])), block_m_result_switch_x[x][block], c=color, linestyle='dashed') #, yerr=block_std_result_switch_x[x][block] 
        axs[block].errorbar(range(len(proba_switch_x[x][block])), block_m_v_anti_switch_x[x][block], c=color) # ,yerr=block_std_v_anti_switch_x[x][block]

        axs[block].set_xticks(np.arange(len(proba_switch_x[x][block])))
        axs[block].set_xticklabels(proba_switch_x[x][block])
        axs[block].set_title('moyenne résultat et anticipation bloc %s'%(block+1), fontsize=14)
        axs[block].set_xlabel('probabilité')

In [ ]:
fig_width= 12
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/1.6180))

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    
    dic_m_result = {}
    dic_m_v_anti = {}
    dic_std_result = {}
    dic_std_v_anti = {}
    
    for block in range(N_blocks):
        #print('block', block)
        for s1 in range(len(proba_switch_x[x][block])) :
            dic_m_result[proba_switch_x[x][block][s1]] = block_m_result_switch_x[x][block][s1]
            dic_m_v_anti[proba_switch_x[x][block][s1]] = block_m_v_anti_switch_x[x][block][s1]
            dic_std_result[proba_switch_x[x][block][s1]] = block_std_result_switch_x[x][block][s1]
            dic_std_v_anti[proba_switch_x[x][block][s1]] = block_std_v_anti_switch_x[x][block][s1]
            #print(s1, proba_switch[block][s1], np.correlate(block_result_switch[block][s1], block_v_anti_switch[block][s1])[0])#[0][1])

    m_res = sorted(dic_m_result.items())
    proba = []
    moyenne_resul = []
    m_anti = sorted(dic_m_v_anti.items())
    moyenne_anti = []

    std_res = sorted(dic_std_result.items())
    std_resul = []
    std_anti = sorted(dic_std_v_anti.items())
    std_v_anti = []
    
    for x in range(len(m_res)):
        proba.append(m_res[x][0])
        moyenne_resul.append(m_res[x][1])
        moyenne_anti.append(m_anti[x][1])
        
        std_resul.append(std_res[x][1])
        std_v_anti.append(std_anti[x][1])
        
    #print("\nproba dans l'ordre = ", proba)
    #print("\ncorrelation dans l'ordre = ", correlation)
    plt.errorbar(proba, moyenne_resul, linestyle='dashed', c=color) # , yerr=std_resul
    plt.errorbar(proba, moyenne_anti, c=color) # , yerr=std_v_anti
    plt.title("moyenne résultat et vitesse d'anticipation", fontsize=14)
    plt.xlabel('probabilité')

In [ ]:
fig_width= 12
fig, axs = plt.subplots(3, 1, figsize=(fig_width, (fig_width*2)/1.6180))
fig.subplots_adjust(hspace=0.3)

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    
    dic = {}
    N_blocks = exp[x]['N_blocks']
    
    for block in range(N_blocks):
        #print('block', block)
        corr = []
        for s1 in range(len(proba_switch_x[x][block])) :
            dic[proba_switch_x[x][block][s1]] = np.correlate(block_result_switch_x[x][block][s1], block_v_anti_switch_x[x][block][s1])[0]
            #print(s1, proba_switch_x[x][block][s1], np.correlate(block_result_switch_x[x][block][s1], block_v_anti_switch_x[x][block][s1])[0])#[0][1])
            corr.append(np.correlate(block_result_switch_x[x][block][s1], block_v_anti_switch_x[x][block][s1])[0])#[1])
        axs[block].plot(corr, c=color)
        axs[block].set_xticks(np.arange(len(proba_switch_x[x][block])))
        axs[block].set_xticklabels(proba_switch_x[x][block])
        axs[block].set_title("corrélation résultat et vitesse d'anticipation bloc %s"%(block+1), fontsize=14)
        axs[block].set_xlabel('probabilité')
        axs[block].set_ylabel('corrélation')

In [ ]:
#fig_width = 20
fig, axs = plt.subplots(1, 1, figsize=(fig_width, (fig_width)/1.6180))

for x, color in enumerate(['black','darkgrey','grey','rosybrown','darkred','red']):
    
    print(exp[x]['observer'], exp[x]['timeStr'], '=',color )
    
    dic_corr = {}
    N_blocks = exp[x]['N_blocks']
    for block in range(N_blocks):
        #print('block', block)
        for s1 in range(len(proba_switch_x[x][block])) :
            dic_corr[proba_switch_x[x][block][s1]] = np.correlate(block_result_switch_x[x][block][s1], block_v_anti_switch_x[x][block][s1])[0]
            #print(s1, proba_switch_x[x][block][s1], np.correlate(block_result_switch_x[x][block][s1], block_v_anti_switch_x[x][block][s1])[0])#[0][1])

    corr = sorted(dic_corr.items())
    proba = []
    correlation = []
    for x in range(len(corr)):
        proba.append(corr[x][0])
        correlation.append(corr[x][1])
    #print("\nproba dans l'ordre = ", proba)
    #print("\ncorrelation dans l'ordre = ", correlation)
    plt.plot(proba, correlation, c=color)
    plt.title("corrélation résultat et vitesse d'anticipation", fontsize=14)
    plt.ylabel('corrélation')
    plt.xlabel('probabilité')